In [32]:
import pandas as pd
import numpy as np
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

In [33]:
file_data = 'data/data.xlsx'

In [34]:
# ==============================================================================
# 1. 优化后的 Feed Concentration 解析函数
# ==============================================================================
def get_feed_concentrations(file_path):
    df_raw = pd.read_excel(file_path, sheet_name='feed conc')
    
    target_mets = {'Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Glc', 'Gln', 'Glu', 'Pyr', 
                   'Gly', 'His', 'Ile', 'Lac', 'Leu', 'Lys', 'Met', 'Nh4', 'Phe', 
                   'Pro', 'Ser', 'Thr', 'Tyr', 'Val'}
    names = df_raw.columns.values
    values = df_raw.iloc[1].values # (跳过单位行)
    
    feed_concs = {}
    for n, v in zip(names, values):
        if isinstance(n, str) and n.strip() in target_mets:
            try:
                feed_concs[n.strip()] = float(v)
            except ValueError:
                pass 
                
    return feed_concs

In [43]:
import pandas as pd
import numpy as np

# ==============================================================================
# 2. 数据处理主逻辑 (计算 Mr/IR) - Simulation Ready 版
# ==============================================================================
def process_bioprocess_data(data_file, feed_concs):
    # 读取数据
    df = pd.read_excel(data_file, sheet_name='data')
    
    # --- 2.1 数据清洗 ---
    if str(df.iloc[0]['Time']).strip() == 'h':
        df = df.drop(0).reset_index(drop=True)
        
    cols_to_numeric = df.columns.drop(['Experiment'])
    df[cols_to_numeric] = df[cols_to_numeric].apply(pd.to_numeric, errors='coerce')
        
    processed_dfs = []
    unique_exps = df['Experiment'].unique()
    
    # 定义代谢物列表 (用于后续列名生成)
    met_list = ['Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Glc', 'Gln', 'Glu', 'Pyr', 
                'Gly', 'His', 'Ile', 'Lac', 'Leu', 'Lys', 'Met', 'Nh4', 'Phe', 
                'Pro', 'Ser', 'Thr', 'Tyr', 'Val']
    
    print(f"开始处理 {len(unique_exps)} 个实验批次...")
    
    for exp_id in unique_exps:
        group = df[df['Experiment'] == exp_id].copy().sort_values('Time').reset_index(drop=True)
        
        # --- 2.2 物理量提取 ---
        V_L = group['V'] / 1000.0  # L
        Feed_Vol_Incr_L = group['feed volume'] / 1000.0 
        Sample_Vol_L = group['sample'] / 1000.0 
        Time = group['Time'].values
        
        res = group[['Time', 'Experiment']].copy()
        res['V_L'] = V_L
        # [关键] 保存原始操作变量，用于未来模拟的输入
        res['Feed_Vol_L'] = Feed_Vol_Incr_L
        res['Sample_Vol_L'] = Sample_Vol_L
        
        # ==========================================
        # 2.3 处理 Biomass (Xv)
        # ==========================================
        # Matlab 逻辑: 先算 Mcell 平衡，最后乘因子
        X_raw = group['X'].values # Mcell/mL
        dw_g_per_mcell = np.where(Time < 100, 2.161e-4, 2.87496e-4) # g/Mcell
        
        # Total Mcell
        mass_X_mcell = X_raw * (group['V'].values) # mL * Mcell/mL = Mcell
        
        # Accum (Mcell)
        accum_X_mcell = np.zeros(len(group))
        accum_X_mcell[0] = mass_X_mcell[0] # Initial Mass
        
        for t in range(1, len(group)):
            # Sample Out (Mcell) = Sample(mL) * X(Mcell/mL)
            mass_out = group['sample'].values[t] * X_raw[t]
            accum_X_mcell[t] = accum_X_mcell[t-1] - mass_out
            
        # Mr (Mcell)
        mr_X_mcell = mass_X_mcell - accum_X_mcell
        
        # [调整] 保存 Accum (转换回 gDW)
        # 注意: 这里 Accum 也需要乘以 dw 因子才能在物理层与其他质量相加减
        # Mr(g) = V*C(g) - Accum(g)
        res['Mr_Xv'] = mr_X_mcell * dw_g_per_mcell
        res['Xv'] = X_raw * dw_g_per_mcell * 1000.0 # g/L
        res['Accum_Xv'] = accum_X_mcell * dw_g_per_mcell # g
        
        # ==========================================
        # 2.4 处理 mAb (Product)
        # ==========================================
        mAb_conc = group['mAb'].values # mg/L
        res['Conc_mAb'] = mAb_conc
        
        mass_mAb = V_L * mAb_conc # mg
        accum_mAb = np.zeros(len(group))
        accum_mAb[0] = 0.0 # Matlab 强制归零
        
        for t in range(1, len(group)):
            mass_out = Sample_Vol_L.iloc[t] * mAb_conc[t]
            accum_mAb[t] = accum_mAb[t-1] - mass_out
            
        res['Mr_mAb'] = mass_mAb - accum_mAb
        res['Accum_mAb'] = accum_mAb # mg
        
        # ==========================================
        # 2.5 处理 Metabolites
        # ==========================================
        for met in met_list:
            feed_c = feed_concs.get(met, 0.0)
            
            if met not in group.columns:
                # 如果缺失，补全为0，防止后续Dataset读取报错
                res[f'Conc_{met}'] = 0.0
                res[f'Mr_{met}'] = 0.0
                res[f'Accum_{met}'] = 0.0
                continue
                
            conc = group[met].values # mM
            
            # Initial Accum
            mass_0 = V_L.iloc[0] * conc[0]
            accum_vec = np.zeros(len(group))
            accum_vec[0] = mass_0
            
            for t in range(1, len(group)):
                # Feed In (mmol) = Feed_Vol(L) * Feed_Conc(mM)
                mass_in = Feed_Vol_Incr_L.iloc[t] * feed_c 
                # Sample Out
                mass_out = Sample_Vol_L.iloc[t] * conc[t]
                
                accum_vec[t] = accum_vec[t-1] + mass_in - mass_out
                
            # Mr 计算
            total_mass = V_L * conc
            mr = total_mass - accum_vec
            
            res[f'Conc_{met}'] = conc
            res[f'Mr_{met}'] = mr
            res[f'Accum_{met}'] = accum_vec # mmol
            
        processed_dfs.append(res)

    df_final = pd.concat(processed_dfs, ignore_index=True)
    return df_final

In [44]:
# 1. 解析 Feed
print("正在解析 Feed 配方...")
feed_concs = get_feed_concentrations(file_data)

# 2. 处理数据
print("正在计算 Accum 和 Mr...")
df_mr = process_bioprocess_data(file_data, feed_concs)

# 3. 保存
output_csv = 'processed_data_IR_final.csv'
df_mr.to_csv(output_csv, index=False)

print("-" * 30)
print(f"处理完成。")
print(f"输出文件: {output_csv}")
print(f"数据维度: {df_mr.shape}")
print(f"包含的列 (前10个): {list(df_mr.columns[:10])}")
print("-" * 30)

# 打印前几行检验 (检查单位数量级是否合理，Mr 应该是 mmol 级别)
print("数据预览 (Br1, Glucose):")
print(df_mr[df_mr['Experiment'] == 1][['Time', 'Conc_Glc', 'Mr_Glc']].head())

正在解析 Feed 配方...
正在计算 Accum 和 Mr...
开始处理 9 个实验批次...
------------------------------
处理完成。
输出文件: processed_data_IR_final.csv
数据维度: (189, 80)
包含的列 (前10个): ['Time', 'Experiment', 'V_L', 'Feed_Vol_L', 'Sample_Vol_L', 'Mr_Xv', 'Xv', 'Accum_Xv', 'Conc_mAb', 'Mr_mAb']
------------------------------
数据预览 (Br1, Glucose):
   Time   Conc_Glc    Mr_Glc
0     0  72.550000  0.000000
1    12  65.198513 -1.827211
2    24  56.939987 -4.015453
3    36  48.140276 -6.477662
4    48  39.084531 -9.140549


In [48]:
def validate_matrix(exp_id, df_mr, feature_names, org_mat_file):

    """
    对比计算矩阵与原始矩阵，打印详细的误差统计。
    """
    # Extract Exp 1
    df_exp = df_mr[df_mr['Experiment'] == exp_id].copy()

    # Total 25: Xv, mAb, + 23 mets
    mr_cols = [col for col in df_mr.columns if col.startswith('Mr_')]

    assert len(mr_cols) == 25

    # Create matrix (25 x 21)
    calculated = df_exp[mr_cols].values.T
    original = sio.loadmat(org_mat_file)['data']['m_r'][0][exp_id-1]

    n_vars = calculated.shape[0]
    
    print(f"{'Feature':<10} | {'Max Abs Err':<12} | {'Max Rel Err':<12} | {'R2 Score':<10} | {'Status'}")
    print("-" * 65)
    
    for i in range(n_vars):
        y_calc = calculated[i, :]
        y_true = original[i, :]
        
        # 1. 绝对误差
        abs_diff = np.abs(y_calc - y_true)
        max_abs_err = np.max(abs_diff)
        
        # 2. 相对误差 (防止除以0)
        with np.errstate(divide='ignore', invalid='ignore'):
            rel_diff = abs_diff / (np.abs(y_true) + 1e-6) # 加个小项防止除0
            max_rel_err = np.max(rel_diff)
            
        # 3. R2 Score (衡量趋势一致性)
        r2 = r2_score(y_true, y_calc)
        
        # 判定
        status = "✅" if r2 > 0.99 else "⚠️"
        if max_rel_err > 0.1 and max_abs_err > 0.1: status = "❌" # 误差超过10%且绝对值不忽略不计
        
        name = feature_names[i] if i < len(feature_names) else f"Var_{i}"
        print(f"{name:<10} | {max_abs_err:.2e}     | {max_rel_err:.2e}     | {r2:.4f}     | {status}")

# 定义变量名列表 (25个)
metabolites_order = ['Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Glc', 'Gln', 'Glu', 'Pyr', 'Gly', 'His', 'Ile', 'Lac', 'Leu', 'Lys', 'Met', 'Nh4', 'Phe', 'Pro', 'Ser', 'Thr', 'Tyr', 'Val']
feature_names = ['Xv', 'mAb'] + metabolites_order

# 运行验证
validate_matrix(exp_id=3, df_mr=df_mr, feature_names=feature_names, org_mat_file='data/data.mat')

Feature    | Max Abs Err  | Max Rel Err  | R2 Score   | Status
-----------------------------------------------------------------
Xv         | 4.44e-16     | 2.63e-16     | 1.0000     | ✅
mAb        | 5.68e-14     | 2.16e-16     | 1.0000     | ✅
Ala        | 1.78e-15     | 3.10e-16     | 1.0000     | ✅
Arg        | 3.55e-15     | 1.27e-14     | 1.0000     | ✅
Asn        | 4.44e-16     | 1.50e-15     | 1.0000     | ✅
Asp        | 4.44e-16     | 3.33e-15     | 1.0000     | ✅
Cys        | 4.44e-16     | 4.89e-15     | 1.0000     | ✅
Glc        | 7.11e-15     | 2.34e-16     | 1.0000     | ✅
Gln        | 3.55e-15     | 1.92e-14     | 1.0000     | ✅
Glu        | 2.22e-16     | 4.87e-15     | 1.0000     | ✅
Pyr        | 3.55e-15     | 1.01e-14     | 1.0000     | ✅
Gly        | 3.55e-15     | 7.29e-16     | 1.0000     | ✅
His        | 8.88e-16     | 1.73e-15     | 1.0000     | ✅
Ile        | 7.11e-15     | 2.63e-14     | 1.0000     | ✅
Lac        | 1.78e-15     | 3.98e-16     | 1.0000     | ✅
L

In [46]:

# from sklearn.decomposition import TruncatedSVD

# # ==============================================================================
# # Step 1: 构建反应相关矩阵 S (PCA / TruncatedSVD)
# # ==============================================================================
# def build_s_matrix(csv_file='processed_data_IR_final.csv', n_components=7):
#     print(f"\n[Step 1] Loading data from {csv_file}...")
#     df = pd.read_csv(csv_file)
    
#     # 1. 定义变量顺序 (严格对应 25 种物质)
#     # Xv, mAb, 23种代谢物
#     met_list = ['Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Glc', 'Gln', 'Glu', 'Pyr', 
#                 'Gly', 'His', 'Ile', 'Lac', 'Leu', 'Lys', 'Met', 'Nh4', 'Phe', 
#                 'Pro', 'Ser', 'Thr', 'Tyr', 'Val']
    
#     # 对应的 Mr 列名
#     mr_cols = ['Mr_Xv', 'Mr_mAb'] + [f'Mr_{m}' for m in met_list]
#     species_cols = ['Xv', 'Conc_mAb'] + [f'Conc_{m}' for m in met_list]
    
#     # 提取数据矩阵 (Samples x Features)
#     # 将所有实验的所有时间点拼接在一起
#     X_mr = df[mr_cols].values
    
#     # 2. 归一化 (Matlab逻辑: 除以最大绝对值)
#     # 这一步是为了让不同量级的物质 (如 Glc vs Trp) 在 PCA 中具有可比性
#     max_vals_mr = np.max(np.abs(X_mr), axis=0)
#     max_vals_mr[max_vals_mr == 0] = 1.0 # 防止除零
    
#     X_norm = X_mr / max_vals_mr
    
#     # 3. 执行非中心化 PCA (TruncatedSVD)
#     # Matlab: pca(..., 'Centered', false)
#     print(f"Executing TruncatedSVD (n_components={n_components})...")
#     svd = TruncatedSVD(n_components=n_components, algorithm='randomized', n_iter=10, random_state=42)
#     svd.fit(X_norm)
    
#     # 4. 验证解释方差
#     total_var = np.sum(svd.explained_variance_ratio_)
#     print(f"PCA Explained Variance Ratio: {total_var:.4%}")
#     if total_var > 0.99:
#         print("✅ PCA Verification Passed (>99%)")
#     else:
#         print("⚠️ Warning: PCA Explained Variance < 99%")
        
#     # 5. 构建 S 矩阵
#     # S = Components.T * Scaling (反归一化)
#     # Python SVD components shape: (n_components, n_features) -> (7, 25)
#     # S shape target: (25, 7)
    
#     # 这里的数学逻辑：
#     # Data ~= Scores @ Components
#     # Data_Real = Data_Norm * MaxVals
#     # Data_Real ~= (Scores) @ (Components * MaxVals)
#     # 所以 S = (Components * MaxVals).T
    
#     S_matrix = (svd.components_ * max_vals_mr[np.newaxis, :]).T
#     print(f"S Matrix Shape: {S_matrix.shape}")
    
#     return S_matrix, df, species_cols, mr_cols

In [49]:
import pandas as pd
import numpy as np
import pickle
from sklearn.decomposition import PCA

# ==============================================================================
# 1. PCA 分析与 S 矩阵构建
# ==============================================================================
def build_reaction_correlation_matrix(csv_file, n_components=7):
    # 读取预处理后的数据
    df = pd.read_csv(csv_file)
    
    # 提取所有 M_r 列 (Reacted Amount)
    met_list = ['Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Glc', 'Gln', 'Glu', 'Pyr', 
                'Gly', 'His', 'Ile', 'Lac', 'Leu', 'Lys', 'Met', 'Nh4', 'Phe', 
                'Pro', 'Ser', 'Thr', 'Tyr', 'Val']
    
    mr_cols = ['Mr_Xv', 'Mr_mAb'] + [f'Mr_{m}' for m in met_list]
    species_cols = ['Xv', 'Conc_mAb'] + [f'Conc_{m}' for m in met_list]
    
    # 提取数据矩阵 (Samples x Features)
    # 这里的 Samples 是所有实验的所有时间点拼接
    X_mr = df[mr_cols].values
    
    # 归一化 (Normalization) - 论文方法：除以最大绝对值
    # Matlab: reacted_masses_pca = reacted_masses ./ max(abs(reacted_masses))
    max_vals = np.max(np.abs(X_mr), axis=0)
    max_vals[max_vals == 0] = 1.0 # 防止除零
    X_norm = X_mr / max_vals
    
    # 执行 PCA
    pca = PCA(n_components=n_components, svd_solver='full')
    pca.fit(X_norm)
    
    # 检查解释方差
    explained_var = np.sum(pca.explained_variance_ratio_)
    print(f"PCA ({n_components} components) 累积解释方差: {explained_var:.4%}")
    
    # 构建 S 矩阵 (Reaction Correlation Matrix)
    # S = PCA_Components.T * Scaling_Factors
    # 维度: (25 features, 7 components)
    # 每一列代表一个“宏观反应” (Macro-Reaction) 的化学计量数向量
    S_matrix = pca.components_.T * max_vals[:, np.newaxis]
    
    return S_matrix, df, species_cols, mr_cols

# 运行
S, df, species_cols, mr_cols = build_reaction_correlation_matrix('processed_data_IR_final.csv')
print("S 矩阵形状:", S.shape)

# Save S matrix for model
with open('s_matrix.pkl', 'wb') as f:
    pickle.dump(S, f)
print("S Matrix saved to s_matrix.pkl")

PCA (7 components) 累积解释方差: 99.9357%
S 矩阵形状: (25, 7)
S Matrix saved to s_matrix.pkl


In [51]:
import torch
from torch.utils.data import Dataset, DataLoader

# ==============================================================================
# Step 2: 构建 PyTorch Dataset
# ==============================================================================
class BioreactorDataset(Dataset):
    def __init__(self, df, species_cols, mr_cols, accum_cols, exp_ids, feed_concs_dict, scaler=None):
        self.data_list = []
        
        # 将 feed_concs_dict 转换为 Tensor 向量 (顺序与 species_cols 对应)
        # species_cols: ['Xv', 'mAb', 'Ala', ...]
        # feed vector: [0, 0, C_Ala, ...]
        feed_conc_vec = []
        for col in species_cols:
            met_name = col.replace('Conc_', '')
            # Xv 和 mAb 的 Feed 浓度通常为 0
            if met_name == 'Xv' or met_name == 'mAb' or met_name == 'Conc_mAb':
                feed_conc_vec.append(0.0)
            else:
                feed_conc_vec.append(feed_concs_dict.get(met_name, 0.0))
        
        self.feed_conc_tensor = torch.tensor(feed_conc_vec, dtype=torch.float32)
        
        for exp in exp_ids:
            group = df[df['Experiment'] == exp].sort_values('Time')
            
            conc = group[species_cols].values.astype(np.float32)
            mr = group[mr_cols].values.astype(np.float32)
            accum = group[accum_cols].values.astype(np.float32) # [新增] 直接读取计算好的 Accum
            
            vol = group['V_L'].values.astype(np.float32).reshape(-1, 1)
            time = group['Time'].values.astype(np.float32).reshape(-1, 1)
            
            # Feed Vol & Sample Vol (用于模拟)
            feed_vol = group['Feed_Vol_L'].values.astype(np.float32).reshape(-1, 1)
            sample_vol = group['Sample_Vol_L'].values.astype(np.float32).reshape(-1, 1)
            
            inputs = np.hstack([conc, vol, time])
            if scaler:
                inputs = scaler.transform(inputs)
            
            self.data_list.append({
                'inputs': torch.tensor(inputs),
                'mr': torch.tensor(mr),
                'accum': torch.tensor(accum), # 用于训练时的物理约束
                'conc': torch.tensor(conc),
                'vol': torch.tensor(vol),
                'time': torch.tensor(time),
                'feed_vol': torch.tensor(feed_vol), # 模拟输入
                'sample_vol': torch.tensor(sample_vol), # 模拟输入
                'feed_conc': self.feed_conc_tensor # 常数向量
            })
            
    def __len__(self): return len(self.data_list)
    def __getitem__(self, idx): return self.data_list[idx]

In [54]:
from sklearn.preprocessing import StandardScaler

def prepare_dataloaders(df, species_cols, mr_cols, feed_concs_dict):
    """
    准备 PyTorch DataLoader，支持训练（Accum GT）和模拟（Feed Strategy）。
    
    Args:
        df: 包含所有数据的 DataFrame
        species_cols: 浓度列名列表 ['Xv', 'Conc_mAb', 'Conc_Glc', ...]
        mr_cols: 反应量列名列表 ['Mr_Xv', 'Mr_mAb', 'Mr_Glc', ...]
        feed_concs_dict: Feed 浓度字典 {'Glc': 42.8, ...}
    """
    print(f"\n[Step 2] Preparing PyTorch Datasets...")
    
    # 0. 自动生成 Accum 列名列表 (必须与 species_cols 顺序一一对应)
    # 规则：Xv -> Accum_Xv, Conc_mAb -> Accum_mAb, Conc_Glc -> Accum_Glc
    accum_cols = []
    for col in species_cols:
        if col == 'Xv':
            accum_cols.append('Accum_Xv')
        elif col == 'Conc_mAb':
            accum_cols.append('Accum_mAb')
        else:
            # 假设其他列都是 'Conc_MetName' 格式
            accum_cols.append(col.replace('Conc_', 'Accum_'))
            
    # 简单校验：确保这些列在 df 中存在
    missing_accum = [c for c in accum_cols if c not in df.columns]
    if missing_accum:
        raise ValueError(f"DataFrame 中缺失以下 Accum 列: {missing_accum}\n请检查 process_bioprocess_data 是否正确保存了 Accum。")

    # 1. 划分数据集 (Fixed Split consistent with Matlab)
    train_exps = [1, 2, 3, 4, 9]
    val_exps = [7]
    test_exps = [5, 6, 8]
    
    print(f"Train Exps: {train_exps}")
    print(f"Val Exps:   {val_exps}")
    print(f"Test Exps:  {test_exps}")
    
    # 2. 计算归一化统计量 (Fit Scaler on Train Data ONLY)
    # 提取所有训练数据用于计算均值和方差
    train_df = df[df['Experiment'].isin(train_exps)]
    
    # Input columns: Species(25) + Vol(1) + Time(1) = 27
    input_data = []
    for exp in train_exps:
        group = train_df[train_df['Experiment'] == exp].sort_values('Time')
        conc = group[species_cols].values
        vol = group['V_L'].values.reshape(-1, 1)
        time = group['Time'].values.reshape(-1, 1)
        input_data.append(np.hstack([conc, vol, time]))
        
    all_train_inputs = np.vstack(input_data)
    
    scaler = StandardScaler()
    scaler.fit(all_train_inputs)
    print("Scaler fitted on training data.")
    
    # 3. 构建 Dataset 对象 (传入 accum_cols 和 feed_concs_dict)
    # 注意：这里的 BioreactorDataset 类必须是你刚才更新过的版本
    train_dataset = BioreactorDataset(df, species_cols, mr_cols, accum_cols, train_exps, feed_concs_dict, scaler)
    val_dataset = BioreactorDataset(df, species_cols, mr_cols, accum_cols, val_exps, feed_concs_dict, scaler)
    test_dataset = BioreactorDataset(df, species_cols, mr_cols, accum_cols, test_exps, feed_concs_dict, scaler)
    
    # 4. 构建 DataLoader
    # batch_size = 4 (Mini-batch training)
    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False) # Val/Test 通常按序列逐个评估
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    
    return train_loader, val_loader, test_loader, scaler

In [55]:
# Run Step 2
train_loader, val_loader, test_loader, scaler = prepare_dataloaders(df, species_cols, mr_cols, feed_concs)


def validate_dataloader(train_loader):
    # Verification
    print("\n[Verification] Checking DataLoader Output:")
    sample_batch = next(iter(train_loader))
    inputs = sample_batch['inputs']
    targets = sample_batch['mr']

    print(f"Input Batch Shape: {inputs.shape} (Batch, Seq, Features)")
    print(f"Target Batch Shape: {targets.shape} (Batch, Seq, Features)")

    if inputs.shape[2] == 27 and targets.shape[2] == 25:
        print("✅ Step 2 Verification Passed: Dimensions correct (In=27, Out=25)")
    else:
        print("❌ Step 2 Verification Failed: Dimensions incorrect")
        
validate_dataloader(train_loader)


[Step 2] Preparing PyTorch Datasets...
Train Exps: [1, 2, 3, 4, 9]
Val Exps:   [7]
Test Exps:  [5, 6, 8]
Scaler fitted on training data.

[Verification] Checking DataLoader Output:
Input Batch Shape: torch.Size([4, 21, 27]) (Batch, Seq, Features)
Target Batch Shape: torch.Size([4, 21, 25]) (Batch, Seq, Features)
✅ Step 2 Verification Passed: Dimensions correct (In=27, Out=25)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pickle
import copy
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader


# ==============================================================================
# 1. 自定义加权损失函数 (Weighted MSE - Paper Eq S1.3)
# ==============================================================================
class WeightedMSELoss(nn.Module):
    def __init__(self, max_vals, device):
        super().__init__()
        # 论文中使用 1/sigma^2 作为权重
        # Matlab 代码实际上使用了 1/max^2 (归一化 MSE)
        # 这里 max_vals 是 25 个物种的最大浓度值
        self.weights = 1.0 / (torch.tensor(max_vals, dtype=torch.float32).to(device) ** 2)
        # 防止权重过大 (比如 max=0 的情况)
        self.weights = torch.clamp(self.weights, max=1e6)
        
    def forward(self, pred, target):
        # pred, target: (Batch, Seq, 25)
        diff = (pred - target) ** 2
        # 按特征维度加权求和，然后对 Batch 和 Seq 取平均
        weighted_diff = diff * self.weights
        return torch.mean(weighted_diff)

# ==============================================================================
# 2. 混合神经 ODE 模型 (Hybrid LSTM)
# ==============================================================================
class HybridLSTM(nn.Module):
    def __init__(self, S_matrix, input_dim=27, hidden_dim=16, latent_dim=7, device='cpu'):
        super().__init__()
        self.device = device
        
        # [关键] 冻结的 S 矩阵 (25, 7)
        self.S = torch.tensor(S_matrix, dtype=torch.float32).to(device)
        self.register_buffer('S_const', self.S) # 不参与梯度更新
        
        # 神经网络: In(27) -> ReLU(16) -> LSTM(7) -> Linear(7)
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU()
        )
        self.lstm = nn.LSTM(input_size=hidden_dim, hidden_size=latent_dim, batch_first=True)
        self.fc_out = nn.Linear(latent_dim, latent_dim) # Output: 7 Scores
        
    def forward(self, x_seq):
        """
        x_seq: (Batch, Seq, 27)
        """
        batch_size, seq_len, _ = x_seq.shape
        
        # 1. NN 前向传播
        # Flatten for Linear
        x_flat = x_seq.reshape(-1, x_seq.size(2))
        features = self.encoder(x_flat)
        features = features.reshape(batch_size, seq_len, -1)
        
        # LSTM
        lstm_out, _ = self.lstm(features)
        
        # Output Scores (Batch, Seq, 7)
        scores = self.fc_out(lstm_out)
        
        # 2. 混合层: 映射回 25 种反应增量
        # dMr = Scores @ S.T
        # (Batch, Seq, 7) @ (7, 25) -> (Batch, Seq, 25)
        delta_mr_reaction = torch.matmul(scores, self.S.t())
        
        return delta_mr_reaction

# ==============================================================================
# 3. 物理层与训练器
# ==============================================================================
class HybridTrainer:
    def __init__(self, model, train_loader, val_loader, max_conc_vals, scaler):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = model.to(self.device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        
        # 优化器 (Adam, lr=0.001 as per paper)
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)
        
        # 损失函数
        self.criterion = WeightedMSELoss(max_conc_vals, self.device)
        
        # 归一化工具 (用于反归一化 Model Input)
        self.scaler_mean = torch.tensor(scaler.mean_, dtype=torch.float32).to(self.device)
        self.scaler_scale = torch.tensor(scaler.scale_, dtype=torch.float32).to(self.device)
        
        self.history = {'train_loss': [], 'val_loss': []}

    def physics_step(self, mr_curr, dMr_pred, accum_next, v_next):
            """
            物理更新方程 (基于质量守恒)
            
            Args:
                mr_curr: 当前时刻的反应累积量 (Batch, 25)
                dMr_pred: NN预测的本步反应增量 (Batch, 25)
                accum_next: 下一时刻的物理累积量 (Batch, 25) [来自Dataset]
                            Accum = Sum(Feed_in) - Sum(Sample_out)
                v_next: 下一时刻的体积 (Batch, 1)
                
            Returns:
                c_next: 下一时刻的浓度 (用于作为下一步NN的输入)
                mr_next: 下一时刻的反应累积量
            """
            # 1. 更新反应累积量 (Reacted Mass State Update)
            mr_next = mr_curr + dMr_pred
            
            # 2. 计算总质量 (Total Mass = Reacted + Physical_Accumulated)
            # 这是论文 S1.3 公式的等价形式，但数值上更稳定
            total_mass_next = mr_next + accum_next
            
            # 3. 计算浓度 (Concentration = Mass / Vol)
            c_next = total_mass_next / v_next
            
            return c_next, mr_next

    def train_epoch(self):
        self.model.train()
        total_loss = 0
        
        for batch in self.train_loader:
            # 数据移动到 GPU
            inputs = batch['inputs'].to(self.device) # (B, T, 27) Scaled
            target_mr = batch['mr'].to(self.device)  # (B, T, 25) Real Mass
            
            # 前向传播 (一次性算出整个序列的 dMr)
            # 输出: (B, T, 25) - 每个时间步的反应增量
            pred_dMr_seq = self.model(inputs)
            
            # --- 序列累积误差计算 ---
            # 我们希望 Pred_Mr(t+1) 接近 Target_Mr(t+1)
            # Pred_Mr(t+1) = Target_Mr(t) + Pred_dMr(t)
            # 这是一种 "Teacher Forcing" 策略 (使用真实的上一步作为基准)
            
            # 取 t=0 到 t=T-1 的 Target 作为基准
            mr_curr = target_mr[:, :-1, :]
            # 取 t=1 到 t=T 的 Target 作为目标
            mr_next_target = target_mr[:, 1:, :]
            
            # 对应的预测增量 (去掉最后一步的预测，因为没有 Target)
            dMr_pred = pred_dMr_seq[:, :-1, :]
            
            # 预测的下一步 Mr
            mr_next_pred = mr_curr + dMr_pred
            
            # 计算 Loss (直接在 Reacted Mass 上计算 WMSE)
            loss = self.criterion(mr_next_pred, mr_next_target)
            
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            
            total_loss += loss.item()
            
        return total_loss / len(self.train_loader)

    def validate(self):
        self.model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in self.val_loader:
                inputs = batch['inputs'].to(self.device)
                target_mr = batch['mr'].to(self.device)
                
                pred_dMr_seq = self.model(inputs)
                
                mr_curr = target_mr[:, :-1, :]
                mr_next_target = target_mr[:, 1:, :]
                dMr_pred = pred_dMr_seq[:, :-1, :]
                
                mr_next_pred = mr_curr + dMr_pred
                loss = self.criterion(mr_next_pred, mr_next_target)
                val_loss += loss.item()
        return val_loss / len(self.val_loader)

    def fit(self, epochs=200, patience=20):
        print(f"Starting training on {self.device}...")
        best_val_loss = float('inf')
        patience_counter = 0
        best_model_wts = copy.deepcopy(self.model.state_dict())
        
        for epoch in range(epochs):
            train_loss = self.train_epoch()
            val_loss = self.validate()
            
            self.history['train_loss'].append(train_loss)
            self.history['val_loss'].append(val_loss)
            
            if epoch % 10 == 0:
                print(f"Epoch {epoch:3d} | Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f}")
            
            # Checkpoint
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model_wts = copy.deepcopy(self.model.state_dict())
                patience_counter = 0
            else:
                patience_counter += 1
                
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch}")
                break
                
        # Load best weights
        self.model.load_state_dict(best_model_wts)
        return self.history

# ==============================================================================
# 4. 执行训练验证
# ==============================================================================
if __name__ == "__main__":
    # 加载 Step 1 的结果
    with open('s_matrix.pkl', 'rb') as f:
        S_matrix = pickle.load(f)
    
    # 重新加载 Step 2 的 DataLoader (如果变量不在内存中)
    # 假设 df, species_cols, mr_cols 已经在内存中，否则需要重新生成
    # 这里为了演示，直接使用上面的 train_loader
    
    # 获取用于 Loss 归一化的 Max Values (从 scaler 中估算或重新计算)
    # 这里我们简单重新计算一下 Training Set 的 Max Mr
    train_df = df[df['ExperimentID'].isin([1, 2, 3, 4, 9])]
    max_mr_vals = np.max(np.abs(train_df[mr_cols].values), axis=0)
    max_mr_vals[max_mr_vals==0] = 1.0
    
    # 初始化模型
    model = HybridLSTM(S_matrix, input_dim=27, latent_dim=7)
    
    # 初始化训练器
    trainer = HybridTrainer(model, train_loader, val_loader, max_mr_vals, scaler)
    
    # 开始训练
    history = trainer.fit(epochs=200)
    
    # 绘制 Loss
    plt.figure(figsize=(10, 5))
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Val Loss')
    plt.yscale('log')
    plt.title('Hybrid Model Training (WMSE)')
    plt.legend()
    plt.show()
    
    print("✅ Step 3 & 4 Verification Passed: Model trained and converged.")